In [1]:
!pip -q install "sentence-transformers>=3.0.0" "transformers>=4.40" faiss-cpu "pymupdf>=1.24" pdfplumber pillow opencv-python "gradio>=4.31" "rank-bm25" "torch>=2.2" "accelerate>=0.30"

# OCR (optional, for scanned PDFs)
!apt -yq update && apt -yq install tesseract-ocr libtesseract-dev
!pip -q install pytesseract

# Clean workspace directories
import os, shutil, pathlib
for p in ["data/raw","data/processed/images","data/processed/text","data/processed/tables","indexes"]:
    pathlib.Path(p).mkdir(parents=True, exist_ok=True)

print("✅ Setup complete.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 86.8 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]


In [2]:
from google.colab import files
import shutil, os

print("📥 Select one or more PDFs from your computer…")
uploaded = files.upload()  # Choose your technical PDFs
for name in uploaded:
    shutil.move(name, os.path.join("data/raw", name))
print("✅ Uploaded to data/raw")


📥 Select one or more PDFs from your computer…


Saving Advanced ChatGPT Prompt Engineering_Mindstream x HubSpot.pdf to Advanced ChatGPT Prompt Engineering_Mindstream x HubSpot.pdf
✅ Uploaded to data/raw


In [3]:
from pathlib import Path
import fitz  # PyMuPDF
import pdfplumber
from PIL import Image
import io, json, re, pytesseract

RAW = Path("data/raw")
OUT_TEXT = Path("data/processed/text")
OUT_IMG = Path("data/processed/images")
OUT_TBL = Path("data/processed/tables")
OUT_TEXT.mkdir(parents=True, exist_ok=True)
OUT_IMG.mkdir(parents=True, exist_ok=True)
OUT_TBL.mkdir(parents=True, exist_ok=True)

def clean_whitespace(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

def extract_pdf(pdf_path: Path):
    doc = fitz.open(pdf_path)
    text_records = []  # list of {doc, page, text}
    figure_records = []  # list of {doc, page, img_path, caption?}

    # ---- Text + Images (and naive caption guess: text near image bbox)
    for pno, page in enumerate(doc):
        # TEXT (layout text)
        page_text = page.get_text("text")
        if not page_text or len(page_text.strip()) < 10:
            # Fallback OCR (rasterize page)
            pix = page.get_pixmap(matrix=fitz.Matrix(2,2))
            img = Image.open(io.BytesIO(pix.tobytes("png")))
            ocr_text = pytesseract.image_to_string(img)
            page_text = ocr_text
        text_records.append({
            "doc": pdf_path.name,
            "page": pno+1,
            "text": clean_whitespace(page_text or "")
        })

        # IMAGES
        for img_index, img in enumerate(page.get_images(full=True), start=1):
            xref = img[0]
            pix = fitz.Pixmap(doc, xref)
            if pix.alpha:  # remove alpha
                pix = fitz.Pixmap(pix, 0)
            img_bytes = pix.tobytes("png")
            img_name = f"{pdf_path.stem}_p{pno+1}_img{img_index}.png"
            img_path = OUT_IMG / img_name
            img_path.write_bytes(img_bytes)

            # naive caption: search text lines near the image vertical region (simple heuristic)
            # (We skip bbox parsing for brevity; you can upgrade later.)
            figure_records.append({
                "doc": pdf_path.name,
                "page": pno+1,
                "image": str(img_path),
                "caption": ""  # placeholder; upgrade later using page.get_text("blocks") with bbox matching
            })

    # ---- Tables (pdfplumber)
    with pdfplumber.open(pdf_path) as pdf:
        for pno, page in enumerate(pdf.pages, start=1):
            try:
                tables = page.extract_tables()
            except Exception:
                tables = []
            for t_idx, table in enumerate(tables, start=1):
                # Save simple CSV
                csv_name = f"{pdf_path.stem}_p{pno}_tbl{t_idx}.csv"
                csv_path = OUT_TBL / csv_name
                # normalize ragged rows
                max_cols = max((len(r) for r in table if r), default=0)
                norm = [ (row + [""]*(max_cols-len(row))) if row else [""]*max_cols for row in table ]
                import csv
                with open(csv_path, "w", newline="", encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerows(norm)

    # Save text as jsonl
    import json, gzip
    jl_path = OUT_TEXT / f"{pdf_path.stem}.jsonl"
    with open(jl_path, "w", encoding="utf-8") as f:
        for rec in text_records:
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

    print(f"✅ Ingested {pdf_path.name}: {len(text_records)} pages, {len(figure_records)} figures")

# Run for all PDFs
for pdf in RAW.glob("*.pdf"):
    extract_pdf(pdf)

print("✅ Ingestion complete.")


✅ Ingested Advanced ChatGPT Prompt Engineering_Mindstream x HubSpot.pdf: 36 pages, 21 figures
✅ Ingestion complete.


In [4]:
import json, faiss, numpy as np, torch
from pathlib import Path
from sentence_transformers import SentenceTransformer, CrossEncoder
from PIL import Image

TEXT_EMB_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
IMG_EMB_MODEL  = "sentence-transformers/clip-ViT-B-32"

text_model = SentenceTransformer(TEXT_EMB_MODEL, device="cuda" if torch.cuda.is_available() else "cpu")
clip_model = SentenceTransformer(IMG_EMB_MODEL, device="cuda" if torch.cuda.is_available() else "cpu")

# ---- Load text chunks
def load_text_chunks():
    chunks = []
    for jl in Path("data/processed/text").glob("*.jsonl"):
        with open(jl, "r", encoding="utf-8") as f:
            for line in f:
                rec = json.loads(line)
                # simple chunking: split by ~800 chars
                txt = rec["text"]
                page = rec["page"]; doc = rec["doc"]
                for i in range(0, len(txt), 800):
                    chunk = txt[i:i+800]
                    if len(chunk.strip()) > 20:
                        chunks.append({
                            "doc": doc, "page": page, "text": chunk
                        })
    return chunks

text_chunks = load_text_chunks()
print(f"Loaded {len(text_chunks)} text chunks")

# ---- Embed text
text_corpus = [c["text"] for c in text_chunks]
text_emb = text_model.encode(text_corpus, batch_size=64, convert_to_numpy=True, show_progress_bar=True, normalize_embeddings=True)

# ---- Build FAISS for text
dim_t = text_emb.shape[1]
index_text = faiss.IndexFlatIP(dim_t)  # using dot with normalized = cosine
index_text.add(text_emb)
faiss.write_index(index_text, "indexes/text.faiss")
np.save("indexes/text_meta.npy", np.array([(c["doc"], c["page"]) for c in text_chunks], dtype=object))

# ---- Build image embeddings
from pathlib import Path
img_paths = sorted(Path("data/processed/images").glob("*.png"))
img_emb = None
if img_paths:
    pil_images = [Image.open(p).convert("RGB") for p in img_paths]
    img_emb = clip_model.encode(pil_images, batch_size=32, convert_to_numpy=True, show_progress_bar=True, normalize_embeddings=True)
    dim_i = img_emb.shape[1]
    index_img = faiss.IndexFlatIP(dim_i)
    index_img.add(img_emb)
    faiss.write_index(index_img, "indexes/images.faiss")
    with open("indexes/images_list.json","w") as f:
        json.dump([str(p) for p in img_paths], f)
    print(f"Built image index with {len(img_paths)} figures")
else:
    print("⚠️ No figures found; image index skipped.")

print("✅ Indexing complete.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loaded 72 text chunks


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Built image index with 21 figures
✅ Indexing complete.


In [7]:
import json, numpy as np, faiss, torch
from sentence_transformers import CrossEncoder
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load indexes & metadata
index_text = faiss.read_index("indexes/text.faiss")
text_meta = np.load("indexes/text_meta.npy", allow_pickle=True)
with open("indexes/images_list.json","r") as f:
    image_list = json.load(f) if Path("indexes/images.faiss").exists() else []
index_img = faiss.read_index("indexes/images.faiss") if Path("indexes/images.faiss").exists() else None

# Models
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2", device="cuda" if torch.cuda.is_available() else "cpu")
gen_tok = AutoTokenizer.from_pretrained("google/flan-t5-base")
gen_lm  = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base").to("cuda" if torch.cuda.is_available() else "cpu")

# Keep text chunks in memory for fast mapping back
# Recreate text_corpus in same order as embeddings were built
def rebuild_text_corpus():
    corpus = []
    ptr = 0
    for jl in Path("data/processed/text").glob("*.jsonl"):
        with open(jl, "r", encoding="utf-8") as f:
            for line in f:
                rec = json.loads(line)
                txt = rec["text"]; page = rec["page"]; doc = rec["doc"]
                for i in range(0, len(txt), 800):
                    chunk = txt[i:i+800].strip()
                    if len(chunk) > 20:
                        corpus.append({"doc": doc, "page": page, "text": chunk})
    return corpus

text_corpus = rebuild_text_corpus()

# Embedders (reuse SentenceTransformer objects)
from sentence_transformers import SentenceTransformer
text_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device="cuda" if torch.cuda.is_available() else "cpu")
clip_model = SentenceTransformer("sentence-transformers/clip-ViT-B-32", device="cuda" if torch.cuda.is_available() else "cpu")

def rrf_fuse(rank_lists, k=60, topn=10):
    scores = {}
    for ranks in rank_lists:
        for rank, idx in enumerate(ranks):
            scores[idx] = scores.get(idx, 0) + 1.0 / (k + rank + 1)
    fused = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [i for i, _ in fused[:topn]]

def search(query, topk_text=15, topk_img=8, fuse_topn=12, rerank_topn=6):
    # text retrieval
    qv = text_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D_t, I_t = index_text.search(qv, topk_text)
    I_t = I_t[0].tolist()

    # image retrieval (CLIP text→image)
    rank_lists = [I_t]
    if index_img is not None and image_list:
        qv_img = clip_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
        D_i, I_i = index_img.search(qv_img, topk_img)
        # map image hits to nearest text chunks on same page/doc (simple heuristic)
        img_hits = []
        for idx in I_i[0]:
            img_path = image_list[idx]
            # find all chunks from same doc+page, take first 2
            m = re.search(r"(.*)_p(\d+)_img\d+\.png$", Path(img_path).stem)
            docstem, page = (m.group(1), int(m.group(2))) if m else (None, None) # Check if m is not None
            # match by docstem in filename to doc (approx)
            if docstem and page: # Only proceed if docstem and page were successfully extracted
                candidates = [i for i, meta in enumerate(text_meta) if (docstem in meta[0]) and meta[1]==page]
                img_hits.extend(candidates[:2])
        rank_lists.append(img_hits)

    # RRF fusion
    fused = rrf_fuse(rank_lists, topn=fuse_topn)

    # Cross-encoder rerank (on fused text candidates)
    pairs = [(query, text_corpus[i]["text"]) for i in fused]
    scores = reranker.predict(pairs)
    reranked = [x for _, x in sorted(zip(scores, fused), key=lambda x: x[0], reverse=True)][:rerank_topn]

    results = []
    for i in reranked:
        rec = text_corpus[i]
        results.append({
            "doc": rec["doc"], "page": rec["page"], "text": rec["text"], "index": i
        })
    return results

def synthesize_answer(query, contexts, max_new_tokens=300):
    # build a grounded prompt
    ctx_block = "\n\n".join([f"[{i+1}] (Doc: {c['doc']}, Page: {c['page']})\n{c['text']}" for i,c in enumerate(contexts)])
    prompt = (
        "You are a precise assistant. Answer the user's question strictly using the evidence provided. "
        "Cite sources in brackets like [1], [2] using the indices of the snippets. "
        "If the answer is not in the context, say you don't know.\n\n"
        f"Question: {query}\n\n"
        f"Context:\n{ctx_block}\n\n"
        "Answer:"
    )
    inputs = gen_tok(prompt, return_tensors="pt", truncation=True).to(gen_lm.device)
    out = gen_lm.generate(**inputs, max_new_tokens=max_new_tokens)
    return gen_tok.decode(out[0], skip_special_tokens=True)

In [9]:
test_query = "Give me table of contents please"
hits = search(test_query)
print(f"Top {len(hits)} snippets:")
for i, h in enumerate(hits, 1):
    print(f"\n[{i}] {h['doc']} p.{h['page']}  —  {h['text'][:300]}…")

print("\n---\nAnswer:\n")
print(synthesize_answer(test_query, hits))


Top 6 snippets:

[1] Advanced ChatGPT Prompt Engineering_Mindstream x HubSpot.pdf p.2  —  Table of Contents 4 Where Are You Now? Go to section → 28 Modular Systems Development Go to section → 32 Custom Framework Development Go to section → 5 Engineering Mindset & Foundations Go to section → DAY 1 8 Advanced Framework Design Go to section → DAY 2 Example-Driven Engineering Go to section →…

[2] Advanced ChatGPT Prompt Engineering_Mindstream x HubSpot.pdf p.21  —  Advanced ChatGPT Prompt Engineering: From Basic to Expert in 7 Days 21 Common Format Control Methods: Table Specifications Present the data in a table with the following columns: - Column 1: [Name] - Column 2: [Description] - Column 3: [Metric 1] - Column 4: [Metric 2] Sort the table by [column] in …

[3] Advanced ChatGPT Prompt Engineering_Mindstream x HubSpot.pdf p.29  —  ided 4. Address limitations and uncertainties explicitly 5. Organize insights from most to least significant Example: Business Writing Format Module Structu

In [12]:
import gradio as gr
from PIL import Image
import base64, io, re, os, json, numpy as np

# Optional: small helper to find figure thumbnails for cited pages
def page_figures(doc_name, page):
    thumbs = []
    for p in Path("data/processed/images").glob("*.png"):
        stem = p.stem
        # match doc stem & page
        if doc_name.replace(".pdf","") in stem and f"_p{page}_" in stem:
            thumbs.append(str(p))
    return thumbs[:3]

def ui_query(q):
    if not q or len(q.strip()) < 3:
        return gr.update(value="Please enter a question."), None, None
    ctx = search(q)
    ans = synthesize_answer(q, ctx)

    # Build HTML with citations and thumbnails
    html = f"<h3>Answer</h3><p>{ans}</p><hr><h4>Evidence</h4>"
    gallery = []
    for i, c in enumerate(ctx, 1):
        snippet = c["text"]
        # truncate & highlight query terms (simple)
        snippet = snippet[:600]
        html += f"<p><b>[{i}]</b> <i>{c['doc']}</i>, page {c['page']}<br>{snippet}</p>"
        thumbs = page_figures(c["doc"], c["page"])
        for t in thumbs:
            gallery.append(t)
    return html, gallery, [f"{c['doc']} (p.{c['page']})" for c in ctx]

with gr.Blocks(title="DocuVision QA — Multimodal RAG") as demo:
    gr.Markdown("## 📄🔎 DocuVision QA — Multimodal RAG over PDFs\nAsk questions about your uploaded PDFs. Answers are grounded with citations.")
    with gr.Row():
        q = gr.Textbox(label="Your Question", placeholder="e.g., What dataset was used and how was it preprocessed?")
    btn = gr.Button("Search")
    out_html = gr.HTML()
    out_gallery = gr.Gallery(label="Related Figures (thumbnails)", columns=3, height="auto")
    out_list = gr.List(label="Citations (Doc, Page)")
    btn.click(fn=ui_query, inputs=q, outputs=[out_html, out_gallery, out_list])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://13b5ee216ceac0bb54.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
